In [ ]:
# 代码参考 https://github.com/bojone/bert4keras/blob/master/pretraining/pretraining.py
# 我只增加了加载优化器参数

# 基本环境配置

In [1]:
# 锁定版本

! pip install tensorflow==2.4
! pip install https://github.com/bojone/bert4keras/archive/3370e32862d419850d34f002ef3dd9e6b704e9e0.zip

     \ 3.4MB 2.0MB/s
     |████████████████████████████████| 378kB 5.3MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
  Created wheel for bert4keras: filename=bert4keras-0.9.7-cp36-none-any.whl size=47969 sha256=cf707e28d4362560c5c3bc142d2163104b82e114caf54ea90ea357d6967c4b31
  Stored in directory: /tmp/pip-ephem-wheel-cache-l_7lbr0t/wheels/9b/7a/e0/48d86836afa73cc40e5504f047855d9d9093cc6fc52f3a0868
Successfully built bert4keras
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [2]:
# 挂载谷歌网盘

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 模型启动

In [3]:
import os, re
os.environ['TF_KERAS'] = '1'  # 必须使用tf.keras

import tensorflow as tf
# 关闭eager模式！！！！极其重要
tf.compat.v1.disable_eager_execution()

# tf.random.set_seed(123456)
# from pretraining.data_utils import *
from bert4keras.models import build_transformer_model
from bert4keras.backend import keras, K
from bert4keras.optimizers import Adam
from bert4keras.optimizers import extend_with_weight_decay
from bert4keras.optimizers import extend_with_layer_adaptation
from bert4keras.optimizers import extend_with_piecewise_linear_lr
from bert4keras.optimizers import extend_with_gradient_accumulation
from keras.layers import Input, Lambda
from keras.models import Model
from keras.callbacks import Callback, CSVLogger

In [12]:
# 模型保存路径
model_saved_path = '/content/gdrive/MyDrive/bert/model_saved_4096/'

if not os.path.exists(model_saved_path):
  os.makedirs(model_saved_path)

# 语料路径 xxxxxxx 为对应的存储分区的路径
corpus_paths = [
    'gs://xxxxxxx/corpus_128.%s.tfrecord' % i for i in range(10)
]

# 其他配置
sequence_length = 128
batch_size = 4096

learning_rate = 5 /  (pow(2, 1.5) * 1e3) # lamb 官方
weight_decay_rate = 0.01
num_warmup_steps = 12500
num_train_steps = 125000
steps_per_epoch = 10000
grad_accum_steps = 8  # 大于1即表明使用梯度累积
epochs = num_train_steps * grad_accum_steps // steps_per_epoch
exclude_from_weight_decay = ['Norm', 'bias']
which_optimizer = 'lamb'  # adam 或 lamb，均自带weight decay
lr_schedule = {
    num_warmup_steps * grad_accum_steps: 1.0,
    num_train_steps * grad_accum_steps: 0.0,
}
floatx = K.floatx()

# 是否加载优化器参数，默认True
load_optimizer_weights = True

In [5]:
# bert_config

config = {
  "attention_probs_dropout_prob": 0.1, 
  "directionality": "bidi", 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "pooler_fc_size": 768, 
  "pooler_num_attention_heads": 12, 
  "pooler_num_fc_layers": 3, 
  "pooler_size_per_head": 128, 
  "pooler_type": "first_token_transform", 
  "type_vocab_size": 2, 
  "vocab_size": 21128
}

In [6]:
# 加载最新参数与参数路径


import re

weights_files = os.listdir(model_saved_path)
weights_files = [(int(re.findall(r"epoch-(\d+)-", x)[0]), os.path.join(model_saved_path, x)) for x in weights_files]
weights_files = sorted(weights_files, key=lambda x:x[1])

initial_epoch = 0
checkpoint_path = None

if len(weights_files) != 0:
  initial_epoch = weights_files[-1][0] + 1
  checkpoint_path = weights_files[-1][1]

print(initial_epoch)
print(checkpoint_path)

0
None


In [16]:
class TrainingDataset(object):
    @staticmethod
    def load_tfrecord(record_names, batch_size, parse_function):
        """加载处理成tfrecord格式的语料
        """
        if not isinstance(record_names, list):
            record_names = [record_names]

        dataset = tf.data.TFRecordDataset(record_names)  # 加载
        dataset = dataset.map(parse_function)  # 解析
        dataset = dataset.repeat()  # 循环
        dataset = dataset.shuffle(batch_size * 1000)  # 打乱
        dataset = dataset.batch(batch_size)  # 成批

        return dataset

class TrainingDatasetRoBERTa(TrainingDataset):
    """预训练数据集生成器（RoBERTa模式）
    """
    @staticmethod
    def load_tfrecord(record_names, sequence_length, batch_size):
        """给原方法补上parse_function
        """
        def parse_function(serialized):
            features = {
                'token_ids': tf.io.FixedLenFeature([sequence_length], tf.int64),
                'mask_ids': tf.io.FixedLenFeature([sequence_length], tf.int64),
            }
            features = tf.io.parse_single_example(serialized, features)
            token_ids = features['token_ids']
            mask_ids = features['mask_ids']
            segment_ids = tf.zeros_like(token_ids, dtype='int64')
            is_masked = tf.not_equal(mask_ids, 0)
            masked_token_ids = K.switch(is_masked, mask_ids - 1, token_ids)
            x = {
                'Input-Token': masked_token_ids,
                'Input-Segment': segment_ids,
                'token_ids': token_ids,
                'is_masked': tf.cast(is_masked, K.floatx()),
            }
            y = {
                'mlm_loss': tf.zeros([1]),
                'mlm_acc': tf.zeros([1]),
            }
            return x, y

        return TrainingDataset.load_tfrecord(
            record_names, batch_size, parse_function
        )

In [17]:
dataset = TrainingDatasetRoBERTa.load_tfrecord(
    record_names=corpus_paths,
    sequence_length=sequence_length,
    batch_size=batch_size // grad_accum_steps,
)

In [18]:
def build_transformer_model_with_mlm():
    """带mlm的bert模型
    """
    bert = build_transformer_model(
        model="bert",
        with_mlm='linear', return_keras_model=False, sequence_length=sequence_length,
        **config
    )

    proba = bert.model.output

    # 辅助输入
    token_ids = Input(shape=(sequence_length,), dtype=tf.int64, name='token_ids')  # 目标id
    is_masked = Input(shape=(sequence_length,), dtype=tf.float32, name='is_masked')  # mask标记

    def mlm_loss(inputs):
        """计算loss的函数，需要封装为一个层
        """
        y_true, y_pred, mask = inputs
        loss = K.sparse_categorical_crossentropy(
            y_true, y_pred, from_logits=True
        )
        loss = K.sum(loss * mask, axis=-1) / (K.sum(mask, axis=-1) + K.epsilon())
        return loss

    def mlm_acc(inputs):
        """计算准确率的函数，需要封装为一个层
        """
        y_true, y_pred, mask = inputs
        y_true = K.cast(y_true, tf.float32)
        acc = keras.metrics.sparse_categorical_accuracy(y_true, y_pred)
        acc = K.sum(acc * mask, axis=-1) / (K.sum(mask, axis=-1) + K.epsilon())
        return acc

    mlm_loss = Lambda(mlm_loss, name='mlm_loss')([token_ids, proba, is_masked])
    mlm_acc = Lambda(mlm_acc, name='mlm_acc')([token_ids, proba, is_masked])

    train_model = Model(
        bert.model.inputs + [token_ids, is_masked], [mlm_loss, mlm_acc]
    )

    def mlm_loss(y_true, y_pred):
      return y_pred

    def mlm_acc(y_true, y_pred):
      return K.stop_gradient(y_pred)

    loss = {
        'mlm_loss': mlm_loss,
        'mlm_acc': mlm_acc,
    }

    return bert, train_model, loss

In [19]:
from bert4keras.optimizers import export_to_custom_objects

# 梯度预归一化 参考 https://developer.nvidia.com/blog/pretraining-bert-with-layer-wise-adaptive-learning-rates/

@export_to_custom_objects
def extend_with_grad_norm(BaseOptimizer):
    """返回新的优化器类，加入梯度预归一化
    """
    class NewOptimizer(BaseOptimizer):
        def __init__(self, *args, **kwargs):
            super(NewOptimizer, self).__init__(*args, **kwargs)

        def _resource_apply(self, grad, var, indices=None):
            op = super(NewOptimizer, self)._resource_apply(K.l2_normalize(grad), var, indices)
            return op

        def get_config(self):
            config = {
                'do_grad_norm':True,
            }
            base_config = super(NewOptimizer, self).get_config()
            return dict(list(base_config.items()) + list(config.items()))

    return NewOptimizer

In [20]:
from tensorflow.python.keras.saving.hdf5_format import load_optimizer_weights_from_hdf5_group
import h5py

def build_transformer_model_for_pretraining():
    """构建训练模型，通用于TPU/GPU
    注意全程要用keras标准的层写法，一些比较灵活的“移花接木”式的
    写法可能会在TPU上训练失败。此外，要注意的是TPU并非支持所有
    tensorflow算子，尤其不支持动态（变长）算子，因此编写相应运算
    时要格外留意。
    """
    bert, train_model, loss = build_transformer_model_with_mlm()


    # 优化器
    optimizer = extend_with_weight_decay(Adam)
    # 梯度预归一化
    # optimizer = extend_with_grad_norm(optimizer)
    
    if which_optimizer == 'lamb':
        optimizer = extend_with_layer_adaptation(optimizer)
    optimizer = extend_with_piecewise_linear_lr(optimizer)
    optimizer_params = {
        'learning_rate': learning_rate,
        'lr_schedule': lr_schedule,
        'weight_decay_rate': weight_decay_rate,
        'exclude_from_weight_decay': exclude_from_weight_decay,
        'exclude_from_layer_adaptation': exclude_from_weight_decay,
        'bias_correction': True,
    }
    if grad_accum_steps > 1:
        optimizer = extend_with_gradient_accumulation(optimizer)
        optimizer_params['grad_accum_steps'] = grad_accum_steps

    optimizer = optimizer(**optimizer_params)


    # 模型定型
    train_model.compile(loss=loss, optimizer=optimizer)

    if checkpoint_path is not None:
      train_model.load_weights(checkpoint_path)

      if load_optimizer_weights:
          with h5py.File(checkpoint_path, mode='r') as f:
              train_model.optimizer._create_all_weights(train_model.trainable_variables)
              optimizer_weight_values = load_optimizer_weights_from_hdf5_group(f)
              
              # 老参数修正梯度
              if len(optimizer_weight_values) < len(train_model.optimizer.weights):
                  # 迭代次数修正
                  optimizer_weight_values[0] = np.array(0)
                  # 增加梯度累计权重
                  for var in train_model.trainable_variables:
                      optimizer_weight_values.append(np.zeros(shape=var.shape))
                  
                  print("参数修复成功")
              
              train_model.optimizer.set_weights(optimizer_weight_values)
              print("优化器参数加载成功")



    return train_model

In [21]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():
    train_model = build_transformer_model_for_pretraining()
    train_model.summary()

Running on TPU  ['10.37.195.18:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.37.195.18:8470
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Querying Tensorflow master (grpc://10.37.195.18:8470) for TPU system metadata.


INFO:tensorflow:Querying Tensorflow master (grpc://10.37.195.18:8470) for TPU system metadata.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 8589934592, -5718637308463307987)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 8589934592, -5718637308463307987)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 7973639068636054144)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 7973639068636054144)


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 128)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     multiple             16226304    Input-Token[0][0]                
                                                                 MLM-Norm[0][0]                   
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 768)     1536        Input-Segment[0][0]        

In [ ]:
class ModelCheckpoint(keras.callbacks.Callback):
    """自动保存最新模型
    """
    def __init__(self, model_saved_path):
        self.model_saved_path = model_saved_path

        if not os.path.exists(self.model_saved_path):
          os.makedirs(self.model_saved_path)
    
    def on_epoch_end(self, epoch, logs=None):
        self.model.save(
          self.model_saved_path + "epoch-%03d-mlm_loss-%.6f-mlm_acc-%.6f.h5" % (epoch, logs['mlm_loss_loss'], logs['mlm_acc_loss']), 
          include_optimizer=True
        )


# 保存模型
checkpoint = ModelCheckpoint(model_saved_path)

# 模型训练
train_model.fit(
    dataset,
    epochs=epochs,
    initial_epoch=initial_epoch,
    steps_per_epoch=steps_per_epoch,
    callbacks=[checkpoint],
)

Instructions for updating:
Use the iterator's `initializer` property instead.


Instructions for updating:
Use the iterator's `initializer` property instead.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:492: UserWarning: `tf.keras.backend.learning_phase_scope` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.learning_phase_scope` is deprecated and '


Epoch 27/100
10000/10000 [==============================] - 8134s 675ms/step - batch: 4999.5000 - size: 1.0000 - num_steps: 1.0000 - loss: 3.2137 - mlm_loss_loss: 2.7654 - mlm_acc_loss: 0.4483
Epoch 28/100
10000/10000 [==============================] - 6548s 653ms/step - batch: 4999.5000 - size: 1.0000 - num_steps: 1.0000 - loss: 3.2766 - mlm_loss_loss: 2.8380 - mlm_acc_loss: 0.4386
Epoch 29/100
10000/10000 [==============================] - 6534s 652ms/step - batch: 4999.5000 - size: 1.0000 - num_steps: 1.0000 - loss: 3.2402 - mlm_loss_loss: 2.7905 - mlm_acc_loss: 0.4497
Epoch 30/100
10000/10000 [==============================] - 6497s 648ms/step - batch: 4999.5000 - size: 1.0000 - num_steps: 1.0000 - loss: 3.1739 - mlm_loss_loss: 2.7155 - mlm_acc_loss: 0.4585
Epoch 31/100
10000/10000 [==============================] - 6457s 644ms/step - batch: 4999.5000 - size: 1.0000 - num_steps: 1.0000 - loss: 3.2160 - mlm_loss_loss: 2.7679 - mlm_acc_loss: 0.4480
Epoch 32/100
10000/10000 [=========